In [ ]:
from pyplasm import *
import numpy 
import sys, os

In [ ]:
sys.setrecursionlimit(1500)

grassTexture = "C:/Users/luca/Desktop/GRAFICA/erba.jpg"
asphaltTexture = "C:/Users/luca/Desktop/GRAFICA/asfalto.jpg"

zero = CUBOID([.0,.0,.0])
baseStruct = STRUCT([zero])
level_height = [30.0,30.0,20.0,30.0,30.0,20.0]
heights = [60.0,20.0,3.5,60.0,20.0]

def readSvg(l,reading_level,path):
   file = open("C:/Users/luca/File-import/level-"+path+".lines","r")
   data = file.read()
   n = countFileDirectory("C:/Users/luca/File-import/")
   file.close()
   d = data.splitlines()
   reading_level = reading_level + [d]
   if l!=n-1:
     return readSvg(l+1,reading_level,path)
   else:
     return reading_level
 

levelBase = readSvg(0,[],"base")
levelStreet = readSvg(0,[],"house")
levelHouse = readSvg(0,[],"street")

def parseLines(l,i, params):
  string_line = params[l][i]
  split_line = string_line.split(",")
  array_line = np.array(split_line, dtype=float)
  return array_line

In [ ]:
def buildGarden(i,s1):
  if i < len(levelBase[0]):
    params = parseLines(0,i,levelBase)
    a_pol = POLYLINE([[params[0],params[1]],[params[2],params[3]]])
    a_off = OFFSET([4.0, 5.5, 5.0])(a_pol)
    s2 = STRUCT([a_off, s1])
    return buildGarden(i+1,s2)
  else:
    s1 = SOLIDIFY(SKEL_2(s1))
    s1 = TEXTURE([grassTexture, TRUE, FALSE, 1, 1, 0, 10, 1])(s1)
    return s1

def buildStreet(i,s1):
  if i < len(levelStreet[0]):
    params = parseLines(0,i,levelStreet)
    a_pol = POLYLINE([[params[0],params[1]],[params[2],params[3]]])
    a_off = OFFSET([0.0, 0.0, 5.0])(a_pol)
    s2 = STRUCT([a_off, s1])
    return buildStreet(i+1,s2)
  else:
    s1=STRUCT([T(3)(5.0),s1])
    return s1

def buildHouseBase(i,s1):
  if i < len(levelHouse[0]):
    params = parseLines(0,i,levelHouse)
    a_pol = POLYLINE([[params[0],params[1]],[params[2],params[3]]])
    a_off = OFFSET([0.0, 0.0, 5.0])(a_pol)
    s2 = STRUCT([a_off, s1])
    return buildHouseBase(i+1,s2)
  else:
    s1 = SOLIDIFY(SKEL_2(s1))
    s1 = TEXTURE([asphaltTexture, TRUE, FALSE, 1, 1, 0, 1, 1])(s1)
    return s1

def buildHouse():
  garden_level = buildGarden(0,baseStruct)
  street_level = buildStreet(0,baseStruct)
  house_level = buildHouseBase(0,baseStruct)
  house=STRUCT([baseStruct,T(3)(0.0),garden_level])
  house=STRUCT([house,T(3)(5.0),street_level])
  house=STRUCT([house,T(3)(5.0),house_level])
  return house

def createCity(i,s1,d1,d2):
  if i < 1:
    h = buildHouse()
    h1=STRUCT([T(1)(d1),h])
    h2=STRUCT([T(1)(d1+1900),h])
    h3=STRUCT([T(2)(d2+2500),h])
    h4=STRUCT([T([1,2])([d1+1900,d2+2500]),h])
    s1= STRUCT([h1,h2,h3,h4,s1])
    s2=STRUCT([T(1)((d1+1900)*2),s1])
    s1= STRUCT([s1,s2])
    return createCity(i+1,s1,d1+1900.0,d2+2500)
  else:
    VIEW(s1)
createCity(0,baseStruct,0.0)